In [5]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import os
import warnings

warnings.filterwarnings('ignore')

In [6]:
# Sprecify the variable name of the industry categories in the CPS Data
ind = 'occ00'

# Specify the mappings of grade completion to years of school attended by the educ variable.
grademappings = {
    'Associate degree-occupational/vocational': 14,
    'HS graduate, GED': 12,
    "Bachelor's degree": 16,
    'Associate degree-academic program': 14,
    '12th grade-no diploma': 12,
    'Some college but no degree': 13,
    'Doctorate': 20,
    '10th grade': 10,
    '11th grade': 11,
    'Professional school': 19,
    "Master's degree": 18,
    '5th-6th grade': 6,
    '7th-8th grade': 8,                              
    '9th grade': 9,
    '1st-4th grade': 4,
    'Less than 1st grade': 0
}

# Runs a regression on the wage column of the dataset. Defined For later
def regression(df):
    y = df['wage']
    X = df.drop(['wage'], axis=1)
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    return model

# Read the data from a stata file
columns_to_read = ['wage', 'union', 'age', ind, 'gradeatn', 'female', 'wbho', 'countyfips']
for i in range(3, 24):
    if i == 3:
        data = pd.read_stata(os.getcwd() + "/epi_cpsbasic_2000_2024/epi_cpsbasic_2003.dta", columns=columns_to_read)
    else:
        num = "{:02d}".format(i)
        temp = pd.read_stata(os.getcwd() + f"/epi_cpsbasic_2000_2024/epi_cpsbasic_20{num}.dta", columns=columns_to_read)
        temp2 = pd.concat([data, temp], ignore_index=True)
        data = temp2

if ind == 'occ00':
    data = data[data[ind] != -1]


cps = data

In [7]:
'''
Returns the cleaned version of the CPS data. This cleaned version only includes workers paid on hourly
wage and who are currently employed. 
'''

def clean_cps(one_hot_encode_industry=True, num_industries=5, keep_industry_col=False): 
    # Drop rows where wage is NaN
    cps_copy = cps.dropna(subset=['wage'])
    cps_copy['wage'] = np.log(cps_copy['wage'])

    # Add a column called educ by enriching another education column
    cps_copy['educ'] = cps_copy['gradeatn'].map(grademappings)

    # Encode the union column to equal 1 for union represented and 0 for not
    # union represented
    cps_copy['union'] =  (cps_copy['union'] == "Union represented").astype(int)

    # Encode the female column to equal 1 for female and 0 for male
    cps_copy['female'] = (cps_copy['female'] == "Female").astype(int)

    # Convert the age column to ints. Remove values outside of 16-79.
    # Square the age column because it has a likely nonlinear with wage.
    cps_copy['age'] = pd.to_numeric(cps_copy['age'], errors='coerce')
    cps_copy.dropna(subset=['age'], inplace=True)
    cps_copy['age'] = cps_copy['age'].astype(int) 
    cps_copy = cps_copy[cps_copy['age'] >= 16]
    cps_copy['age^2'] = (cps_copy['age']**2)

    # One-Hot encode the race column that reports white, hispanic, or black.
    # Drop the column for white because it has the largest proportion of samples
    # Drop the column for other for simplicity
    # remaining columns are "black" and "hispanic"
    cps_copy = pd.get_dummies(cps_copy, columns=['wbho'], drop_first=True, dtype=int)
    cps_copy = cps_copy.drop(['wbho_Other'], axis=1)
    cps_copy = cps_copy.rename(columns={'wbho_Black':'black', 'wbho_Hispanic':'hispanic'})

    # One Hot Encode the Industries
    if one_hot_encode_industry:
        # Pick the top ten most common industries and one-hot encode them.
        num_industries = len(cps_copy[ind].unique())
        industries = cps_copy[ind].value_counts().head(num_industries).index.to_list()
        #industries = industries[:num_industries]
        print(f"The following industry is assumed to be the default: {industries[1]}") #Assumed to be standard
        print(industries)
        cps_copy = cps_copy[cps_copy[ind].isin(industries)]
        cps_copy[ind] = cps_copy[ind].astype('category')
        cps_copy[ind] = cps_copy[ind].cat.remove_unused_categories()
        cps_copy = pd.get_dummies(cps_copy, columns=[ind], prefix_sep='_', prefix='Industry', drop_first=True, dtype=int)

    # include counties
    cps_copy = pd.get_dummies(cps_copy, columns=['countyfips'], prefix_sep='_', prefix='County', drop_first=True, dtype=int)


    # Select specific columns fromt he dataframe
    #cps2023 = cps2023.iloc[:, [0, 2]]
    selected_columns = ['wage', 'union', 'educ', 'age^2', 'female', 'black', 'hispanic'] + [col for col in cps_copy.columns if 'Industry' in col] + [col for col in cps_copy.columns if 'County' in col]
    if keep_industry_col:
        selected_columns.append(ind)

    cps_copy = cps_copy[selected_columns]
    
    return cps_copy




In [8]:
clean = clean_cps(one_hot_encode_industry=False)
print(regression(clean).summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.298
Model:                            OLS   Adj. R-squared:                  0.298
Method:                 Least Squares   F-statistic:                 1.174e+04
Date:                Fri, 26 Apr 2024   Prob (F-statistic):               0.00
Time:                        12:37:15   Log-Likelihood:            -2.7690e+06
No. Observations:             3381174   AIC:                         5.538e+06
Df Residuals:                 3381051   BIC:                         5.540e+06
Df Model:                         122                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     1.43

In [17]:
cps_copy = clean_cps(one_hot_encode_industry=False, keep_industry_col=True)

#all_industries = cps_copy[ind].unique().to_list()
all_industries = cps_copy[ind].unique()
count = 0
curr = 0
for industry in all_industries:
    temp = cps_copy[cps_copy[ind] == industry]
    participation = np.average(temp['union'])
    selected_columns = ['wage', 'union', 'educ', 'age^2', 'female', 'black', 'hispanic'] + [col for col in temp.columns if 'County' in col]
    temp = temp[selected_columns]
    if temp.shape[0] >= 2000: #Only include occupations with more than 200 people. 
        reg = regression(temp)
        insert = reg.params.to_dict()
        insert2 = reg.bse.to_dict()
        insert2 = {("std_err_"+key):value for key, value in insert2.items()}
        insert = {**insert, **insert2}
        insert['industry'] = industry
        insert['percentage of union workers'] = participation
        insert_df =  pd.DataFrame([insert])
        if count==0:
            curr = pd.DataFrame({key: [value] for key, value in insert.items()})
        elif participation >= 0.01: # Only include industries where more than 1% of employeees are in labor unions. 
            temp = pd.concat([curr, insert_df], ignore_index=True)
            curr = temp
    count += 1
curr['union_significant'] = (curr['union']/curr['std_err_union'] > 2)
#curr = curr[curr['significant']==True]
curr = curr[['industry', 'percentage of union workers', 'union', 'union_significant']]
curr = curr.sort_values(by="percentage of union workers", ascending=False)
curr = curr.reset_index(drop=True)
curr = curr.sort_values(by="union", ascending=False)
results = curr
results.head()

,industry,percentage of union workers,union,union_significant
20,Couriers and messengers,0.234793,0.409106,True
40,Carpenters,0.163245,0.337342,True
63,"Painters, construction and maintenance",0.108511,0.303911,True
46,Construction laborers,0.134069,0.291255,True
85,Grounds maintenance workers,0.074985,0.285223,True


$log(\text{wage}_i) = \alpha \text{union}_i + \delta _1\text{educ}_i + \delta _2 \text{age}^2_i + \delta _3 \text{female}_i + \delta _4\text{black}_i + \delta _4 \text{hispanic}_i + \sum_{n_i \in \text{counties}} \delta _\text{county} n_i$

$log(\text{wage}_i) = \alpha \text{union}_i + \delta _1\text{educ}_i + \delta _2 \text{age}^2_i + \delta _3 \text{female}_i + \delta _4\text{black}_i + \delta _4 \text{hispanic}_i$

In [15]:
# Finding the proportion of labor union participation for each occupation
props_map = {}
props = []
inds = []
proportions = clean_cps(one_hot_encode_industry=False, keep_industry_col=True)
all_industries = proportions[ind].unique()
for industry in all_industries:
    temp = proportions[proportions[ind] == industry]
    if temp.shape[0] >= 2000:
        inds.append(industry)
        props.append(np.average(temp['union']))
props_map = {'Industry':inds, 'Proportion of Employees Represented By a Union': props}
proportion_table = pd.DataFrame(props_map)
proportion_table = proportion_table.sort_values(by='Proportion of Employees Represented By a Union', ascending=False)